In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default, .output_wrapper .ui-dialog-titlebar { display: none;}
div[tabindex="0"] {resize:none !important; border:0 !important}
.mpl-message.pull-right {display:none}
label.widget-label {display:none} # Make slider label disappear
</style>

In [ ]:
%matplotlib notebook

from aegis.help.container import Container

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import ipywidgets

matplotlib.rcParams.update({"font.size": 8})

with open("paths.txt", "r") as f:
    paths = f.read().split()[::-1]

dropdown = ipywidgets.Dropdown(
    options=paths,
    value=paths[0],
    description="Load from: ",
    layout=ipywidgets.Layout(width="80%"),
)
dropdown

In [ ]:
print(f"Attempting to load from {dropdown.value}")
container = Container(dropdown.value)
phenotypes = container.get_df("phenotypes")
age_at_birth = container.get_df("age_at_birth")
age_at_genetic = container.get_df("age_at_genetic")
age_at_overshoot = container.get_df("age_at_overshoot")
genotypes = container.get_df("genotypes")
if container.paths:
    print("Loading successful 🙂")
else:
    print("Loading unsuccessful 😢")

In [ ]:
plt.close("all")

nrows = 2
ncols = 3
xlim = (1, 50)
xticks = [1, 10, 20, 30, 40, 50]

max_age = 50
ages = np.arange(1, max_age + 1)


def get_mortality_rate(t):
    return 1 - phenotypes.iloc[t, :max_age]


def get_survivorship(t):
    return phenotypes.iloc[t, :max_age].cumprod()


def get_fertility_rate(t):
    return phenotypes.iloc[t, max_age:]


def get_offspring_number(t):
    return np.cumsum(
        np.multiply(
            list(phenotypes.iloc[t, max_age:]), list(phenotypes.iloc[t, :max_age])
        )
    )


def get_birth_structure(t):
    return age_at_birth.iloc[t]


def get_death_structure(t):
    pseudocount = 0
    return (age_at_genetic.iloc[t] + pseudocount) / (
        age_at_overshoot.iloc[t] + age_at_genetic.iloc[t] + pseudocount
    )


fig = plt.figure(figsize=(9, 6))
fig.canvas.toolbar_visible = False
fig.suptitle(dropdown.value.split("/")[-1])

# PANEL 1
ax1 = fig.add_subplot(nrows, ncols, 1)
ax1.set(
    #     title="💀",
    xlabel="age",
    ylabel="mortality rate",
    ylim=(-0.02, 1.02),
    xlim=xlim,
    xticks=xticks,
)
(ax11,) = ax1.plot(ages, get_mortality_rate(0), c="dodgerblue")
ax12 = ax1.scatter(ages, get_mortality_rate(0), s=10, c="dodgerblue")

# PANEL 2
ax2 = fig.add_subplot(nrows, ncols, 2)
ax2.set(
    xlabel="age",
    ylabel="survivorship",
    ylim=(-0.02, 1.02),
    xlim=xlim,
    xticks=xticks,
)
(ax21,) = ax2.plot(ages, get_survivorship(0), c="dodgerblue")
ax22 = ax2.scatter(ages, get_survivorship(0), s=10, c="dodgerblue")


# PANEL 3
ax3 = fig.add_subplot(nrows, ncols, 4)
ax3.set(
    xlabel="age",
    ylabel="fertility rate",
    ylim=(-0.02, 0.52),
    xlim=xlim,
    xticks=xticks,
)
(ax31,) = ax3.plot(ages, get_fertility_rate(0), c="crimson")
ax32 = ax3.scatter(ages, get_fertility_rate(0), s=10, c="crimson")

# PANEL 4
ax4 = fig.add_subplot(nrows, ncols, 5)
ax4.set(
    xlabel="age",
    ylabel="age-cumulative # of offspring per individual",
    xlim=xlim,
    xticks=xticks,
)
(ax41,) = ax4.plot(ages, get_offspring_number(0), c="crimson")
ax42 = ax4.scatter(ages, get_offspring_number(0), s=10, c="crimson")

# PANEL 5
ax = fig.add_subplot(nrows, ncols, 6)
ax.set(
    xlabel="age",
    ylabel="# of offspring born to parents of age x",
    xlim=xlim,
    ylim=(0, age_at_birth.max().max()),
    xticks=xticks,
)
(ax51,) = ax.plot(ages, get_birth_structure(0), c="crimson")
ax52 = ax.scatter(ages, get_birth_structure(0), s=10, c="crimson")

# PANEL 6
ax = fig.add_subplot(nrows, ncols, 3)
ax.set(
    title="cause of death",
    xlabel="age",
    ylabel="proportion genetic",
    xlim=xlim,
    ylim=(-0.01, 1.02),
    xticks=xticks,
)
ax62 = ax.scatter(ages, get_death_structure(0), s=10, c="dodgerblue")


def update(t):
    # PANEL 1
    ax11.set_ydata(get_mortality_rate(t - 1))
    ax12.set_offsets(np.c_[ages, get_mortality_rate(t - 1)])
    # PANEL 2
    ax21.set_ydata(get_survivorship(t - 1))
    ax22.set_offsets(np.c_[ages, get_survivorship(t - 1)])
    # PANEL 3
    ax31.set_ydata(get_fertility_rate(t - 1))
    ax32.set_offsets(np.c_[ages, get_fertility_rate(t - 1)])
    # PANEL 4
    ax41.set_ydata(get_offspring_number(t - 1))
    ax42.set_offsets(np.c_[ages, get_offspring_number(t - 1)])
    # PANEL 5
    ax51.set_ydata(get_birth_structure(t - 1))
    ax52.set_offsets(np.c_[ages, get_birth_structure(t - 1)])
    # PANEL 6
    ax62.set_offsets(np.c_[ages, get_death_structure(t - 1)])
    # redraw
    fig.canvas.draw_idle()


slider = ipywidgets.IntSlider(
    value=1,
    min=1,
    max=len(phenotypes),
    step=1,
    description=" ",
    layout=ipywidgets.Layout(width="50%"),
)

ipywidgets.interact(
    update,
    t=slider,
)

play = ipywidgets.Play(min=0, max=len(phenotypes), step=1, interval=25)
ipywidgets.interact(
    update,
    t=play,
)

ipywidgets.jslink((play, "value"), (slider, "value"))

plt.tight_layout()
plt.show()
